In [1]:
import numpy as np
import pandas as pd
from itertools import chain

In [2]:
def flatten(multi_list):
    """Mengembalikan list multi dimensi ke dalam list satu dimensi.
    
    Input list yang masuk di proses menggunakan method from_iterable dari
    package itertools dan mengembalikan object berupa generator yang iterable.
    
    Args:
        multi_list: list multi dimensi.
    
    Return:
        list satu dimensi yang nantinya digunakan untuk pemrosesan lebih lanjut.
    """
    
    return chain.from_iterable(multi_list)

In [3]:
def get_data(filename):
    """Fungsi untuk melakukan load data pada file .tsv maupun .txt
    
    File yang digunakan yaitu file .tsv maupun file yang mampu dibuka
    menggunakan teks editor. Dalam file tersebut terdapat tag pembuka <kalimat>
    dan diakhiri tag penutup </kalimat> yang berfungsi sebagai penanda bahwa
    entitas tersebut termasuk ke dalam satu kalimat.
    
    Args:
        filename: string dari nama file yang akan diload datanya.
        
    Return:
        list dari kata-kata dan tags dengan index yang menunjukkan posisi kalimat tersebut.
    """
    
    # Load data dan buka sebagai file
    sentences = []
    tags = []
    with open(filename) as file:
        contents = file.readlines()

    # Hapus karakter \n yang tidak dibutuhkan
    contents = [content.strip() for content in contents]
    idx = 0
    while idx < len(contents):
        word = []
        tag = []
        # looping sampai menemukan pattern dengan awalan </kalimat
        while not contents[idx].startswith('</kalimat'):
            # kondisi jika menemukan sebuah data yang tidak memiliki awalan <kalimat
            if not contents[idx].startswith('<kalimat'):
                temp_word, temp_tag = contents[idx].split("\t")
                word.append(temp_word.lower())
                tag.append(temp_tag)
            idx += 1
        sentences.append(word)
        tags.append(tag)
        idx += 2
        
    return sentences, tags

In [4]:
file_path = "../data/"
file_name = "Indonesian_Manually_Tagged_Corpus_ID.tsv"
words, tags = get_data(file_path + file_name)
words = list(flatten(words))[:1000]
tags = list(flatten(tags))[:1000]

In [5]:
def get_emission_table(words, tags):
# Membuat representasi tabel emission probability dari HMM
    hidden_state = {}
    for word in words:
        # Jika word belum pernah ditemui maka akan digenerate
        # tagset berserta probabilitasnya
        if word not in hidden_state.keys():
            word_tags = []
            for idx, wrd in enumerate(words):
                if wrd == word :
                    word_tags.append(tags[idx])
            # Membuat dictionary berisi semua tagset dari sebauh word
            # pada korpus. Berserta jumlah kemunculan setiap tagsetnya
            tag_count = {}
            for tag in word_tags:
                if tag not in tag_count.keys():
                    tag_count[tag] = 1
                else:
                    tag_count[tag] += 1
            total = 0
            # Mengubah jumlah kemunculan menjadi probabilitas
            tag_prob = {}
            for count in tag_count.values():
                total += count
            for tagset in tag_count.keys():
                tag_prob[tagset] = tag_count[tagset]/total
            hidden_state[word] = tag_prob
        else:
            # Jika word sudah pernah ditemui maka akan dilewati
            continue
    return(hidden_state)

In [6]:
hidden = get_emission_table(words, tags)

In [8]:
len(hidden)

484